In [ ]:
import pandas as pd

data = pd.read_csv("/mnt/c/Users/POOYA/Desktop/data.csv")

data

In [ ]:

def convertAB(Year_AB_1, Year_AB_2, Month_AB, Day_AB, Houre, Minute, Second):
    if Month_AB <=6 and Month_AB >=1:
        if Year_AB_2 >= 20:
            Year = Year_AB_2 + 1300
        else:
            Year = Year_AB_2 + 1400

        Month = Month_AB
        Day = Day_AB
    elif Month_AB <=12 and Month_AB >=7:
        Year = Year_AB_1
        Month = Month_AB
        Day = Day_AB
    else:
        Year = None
        Month = None
        Day = None
    return [int(Year), int(Month), int(Day), int(Houre), int(Minute), int(Second)]



b6 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_6', 'BARF_6', 'AB_BARF_6']]
b6['Houre'] = 6
b6['Minute'] = 30
b6['Second'] = 0
b6['JAM_BARAN'] = b6['BARAN_6'] + b6['AB_BARF_6']

b6[['Date']] = b6.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b6 = b6[['stationCode', 'Date', 'BARAN_6', 'BARF_6', 'AB_BARF_6', 'JAM_BARAN']]
b6.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']



b18 = data[['stationCode', 'Year_AB_1', 'Year_AB_2', 'Month_AB', 'Day_AB', 'BARAN_18', 'BARF_18', 'AB_BARF_18']]
b18['Houre'] = 18
b18['Minute'] = 30
b18['Second'] = 0
b18['JAM_BARAN'] = b18['BARAN_18'] + b18['AB_BARF_18']

b18[['Date']] = b18.apply(
    lambda row : convertAB(row["Year_AB_1"], row["Year_AB_2"], row["Month_AB"], row["Day_AB"], row["Houre"], row["Minute"], row["Second"]),
     axis=1
    )

b18 = b18[['stationCode', 'Date', 'BARAN_18', 'BARF_18', 'AB_BARF_18', 'JAM_BARAN']]
b18.columns = ['stationCode', 'DATE', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']




data = pd.concat([b6, b18])
data[['YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND']] = pd.DataFrame(data.DATE.tolist(), index= data.index)
data['DATE'] = data['YEAR'].astype(str) + "-" + data['MONTH'].astype(str).str.zfill(2) + "-" + data['DAY'].astype(str).str.zfill(2) + " " + data['HOURE'].astype(str).str.zfill(2) + ":" + data['MINUTE'].astype(str).str.zfill(2) + ":" + data['SECOND'].astype(str).str.zfill(2) 





data = data[['stationCode', 'DATE', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND', 'BARAN', 'BARF', 'AB_BARF', 'JAM_BARAN']]


data.sort_values(by=['stationCode', 'YEAR', 'MONTH', 'DAY', 'HOURE', 'MINUTE', 'SECOND'], inplace=True)



In [ ]:
data

In [ ]:
data.to_csv("data.csv", index=False)

In [ ]:
import pandas as pd
import sqlite3
db_path_precipitation = 'App/dashApp/precipitation/precipitation.sqlite'
db_precipitation = sqlite3.connect(db_path_precipitation, check_same_thread=False)





date = "1398/3/1"
time = "18:30:00"
data = [
    {'stationName': 'فریمان', 'baran': 0, 'barf': 0, 'ab_barf': 0}, 
    {'stationName': 'امور آب سبزوار', 'baran': 0, 'barf': 0, 'ab_barf': 0}, 
    {'stationName': 'چناران', 'baran': 0, 'barf': 0, 'ab_barf': 0},
]

In [ ]:
        exist_precipitation_data = pd.read_sql_query(sql="SELECT * FROM precipitation", con=db_precipitation)
        exist_precipitation_data_columns = list(exist_precipitation_data.columns)
        exist_precipitation_data['DATE'] = exist_precipitation_data['YEAR'].astype(str) + "-" + exist_precipitation_data['MONTH'].astype(str).str.zfill(2) + "-" + exist_precipitation_data['DAY'].astype(str).str.zfill(2) + " " + exist_precipitation_data['HOURE'].astype(str).str.zfill(2) + ":" + exist_precipitation_data['MINUTE'].astype(str).str.zfill(2) + ":" + exist_precipitation_data['SECOND'].astype(str).str.zfill(2)
        exist_precipitation_data["uniqueCode"] = exist_precipitation_data["stationCode"].astype(str) + "-" + exist_precipitation_data["DATE"]

In [ ]:
exist_station = pd.read_sql_query(sql="SELECT * FROM station", con=db_precipitation)

In [ ]:
exist_station

In [ ]:
df = pd.DataFrame(columns=exist_precipitation_data_columns[1:] + ["stationName"])

date = date.split("/")
time = time.split(":")

for i in range(len(data)):
    df.loc[i,"stationCode"] = exist_station[exist_station['stationName'] == data[i]["stationName"]].iloc[0,1]
    df.loc[i,"YEAR"] = date[0]
    df.loc[i,"MONTH"] = date[1].zfill(2)
    df.loc[i,"DAY"] = date[2].zfill(2)
    df.loc[i,"HOURE"] = time[0].zfill(2)
    df.loc[i,"MINUTE"] = time[1].zfill(2)
    df.loc[i,"SECOND"] = time[2].zfill(2)
    df.loc[i,"BARAN"] = data[i]["baran"]
    df.loc[i,"BARF"] = data[i]["barf"]
    df.loc[i,"AB_BARF"] = data[i]["ab_barf"]
    df.loc[i,"JAM_BARAN"] = data[i]["baran"] + data[i]["ab_barf"]
    df.loc[i,"stationName"] = data[i]["stationName"]

df['DATE'] = df['YEAR'].astype(str) + "-" + df['MONTH'].astype(str) + "-" + df['DAY'].astype(str) + " " + df['HOURE'].astype(str) + ":" + df['MINUTE'].astype(str) + ":" + df['SECOND'].astype(str)

df["uniqueCode"] = df["stationCode"].astype(str) + "-" + df["DATE"]


In [ ]:
df

In [ ]:
df.loc[df.duplicated(subset=["stationName"]),:]["stationName"].unique().tolist()

In [ ]:
df["uniqueCode"].isin(exist_precipitation_data["uniqueCode"]).any()

In [ ]:
df.loc[df["uniqueCode"].isin(exist_precipitation_data["uniqueCode"]), :]

In [ ]:
from collections import Counter
a = list((Counter(df["uniqueCode"]) - Counter(exist_precipitation_data["uniqueCode"])).elements())
a

In [ ]:
list(set(list(df["uniqueCode"])) - set(a))

In [ ]:
a

In [ ]:
list(df["uniqueCode"])

In [ ]:
df

In [ ]:
df.drop(['uniqueCode', 'DATE', 'stationName'], axis=1)

In [ ]:
exist_precipitation_data

DAILY DATA

In [ ]:
import pandas as pd

data = pd.read_csv("data1.csv")

persian_month = {
    'فروردین': 1,
    'اردیبهشت': 2,
    'خرداد': 3,
    'تیر': 4,
    'مرداد': 5,
    'شهریور': 6,
    'مهر': 7,
    'آبان': 8,
    'آذر': 9,
    'دی': 10,
    'بهمن': 11,
    'اسفند': 12,
}

data["MONTH"] = data["MONTH"].map(persian_month)

data

In [ ]:
data = pd.melt(data, id_vars=["YEAR", "MONTH", "DAY", "HOURE", "MINUTE", "SECOND"], var_name="stationCode", value_name="JAM_BARAN")
data['stationCode'] = data['stationCode'].astype(int)
data["BARAN"] = data["JAM_BARAN"]
data["BARF"] = 0.0
data["AB_BARF"] = 0.0
data = data[["stationCode", "YEAR", "MONTH", "DAY", "HOURE", "MINUTE", "SECOND", "BARAN", "BARF", "AB_BARF", "JAM_BARAN"]]
data

data.to_csv('data.csv', index=False)

In [ ]:
import pandas as pd
data = pd.read_excel("data.xlsx")

type(sorted(list(data[YEAR].unique()))[-2])

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

YEAR = "YEAR"

data = pd.read_excel("data.xlsx")

START_DURATION = 1370
END_DURATION = 1399
START_YEAR = 1400
PREVIOUS_YEAR = sorted(list(data[YEAR].unique()))[-2]

graph1 = go.Figure()

for st in data["stationCode"].unique():

    df = data[data["stationCode"] == st]

    df_duration = df[(df[YEAR] >= START_DURATION) & (df[YEAR] <= END_DURATION)]

    graph1.add_trace(
        go.Box(
            x = df_duration.stationName,
            y = df_duration.JAM_BARAN,
            showlegend=False,
            boxmean=True,
        )
    )


graph1.add_trace(
    go.Scatter(
        x = data.loc[data[YEAR] == PREVIOUS_YEAR, "stationName"],
        y = data.loc[data[YEAR] == PREVIOUS_YEAR, "JAM_BARAN"],
        marker = dict(size=12, color="blue"),
        mode='markers',
        showlegend=True,
        name=PREVIOUS_YEAR
    )
)

graph1.add_trace(
    go.Scatter(
        x = data.loc[data[YEAR] == START_YEAR, "stationName"],
        y = data.loc[data[YEAR] == START_YEAR, "JAM_BARAN"],
        marker = dict(size=12, color="red"),
        mode='markers',
        showlegend=True,
        name=START_YEAR,
        hovertemplate="%{y}<extra></extra>",
    )
)



graph1.update_layout(
    margin={'l': 0, 'r': 0},
    xaxis_title="",
    yaxis_title="بارندگی - میلیمتر",
    autosize=False,
    font=dict(
        family="Tanha-FD",
        size=12,
        color="RebeccaPurple"
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.005,
        xanchor="center",
        x=0.500
    ),
    title=dict(
        text='نمودار جعبه‌ای بارندگی',
        yanchor="top",
        y=0.95,
        xanchor="center",
        x=0.500
    )
)


graph1.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

YEAR = "YEAR"

data = pd.read_excel("data.xlsx")

START_DURATION = 1370
END_DURATION = 1399
START_YEAR = 1400
PREVIOUS_YEAR = sorted(list(data[YEAR].unique()))[-2]

df_mean_duration = pd.pivot_table(
    data[(data[YEAR] >= START_DURATION) & (data[YEAR] <= END_DURATION)],
    values=['JAM_BARAN'],
    index=["stationName"],
    aggfunc=np.mean
).reset_index().round(1)

graph2 = go.Figure()

graph2.add_trace(
    go.Bar(
        x = data.loc[data[YEAR] == START_YEAR, "stationName"],
        y = data.loc[data[YEAR] == START_YEAR, "JAM_BARAN"],
        showlegend=True,
        name=str(START_YEAR),
        hoverinfo=None
    )
)

graph2.add_trace(
    go.Bar(
        x = data.loc[data[YEAR] == PREVIOUS_YEAR, "stationName"],
        y = data.loc[data[YEAR] == PREVIOUS_YEAR, "JAM_BARAN"],
        showlegend=True,
        name=str(PREVIOUS_YEAR),
        hoverinfo=None
    )
)

graph2.add_trace(
    go.Bar(
        x = df_mean_duration.loc[:, "stationName"],
        y = df_mean_duration.loc[:, "JAM_BARAN"],
        showlegend=True,
        name=f"{START_DURATION} - {END_DURATION}",
        text=df_mean_duration.loc[:, "JAM_BARAN"],
        textposition='outside',
        hovertemplate="%{y} : %{x}<extra></extra>",
    )
)



graph2.update_layout(
    margin={'l': 0, 'r': 0},
    xaxis_title="",
    yaxis_title="بارندگی - میلیمتر",
    autosize=False,
    font=dict(
        family="Tanha-FD",
        size=12,
        color="RebeccaPurple"
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.005,
        xanchor="center",
        x=0.500
    ),
    title=dict(
        text='نمودار جعبه‌ای بارندگی',
        yanchor="top",
        y=0.95,
        xanchor="center",
        x=0.500
    )
)


graph2.show()

In [ ]:
import plotly.graph_objects as go

data = pd.read_excel("data.xlsx")

data["occure_precip"] = [1 if x >= 2 else 0 for x in data["JAM_BARAN"]]

In [ ]:
data

In [14]:
import dash_html_components as html
import dash_leaflet as dl
from dash import Dash
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
markers = [dl.Marker(position=[56, 10]), dl.CircleMarker(center=[55, 10])]
polygon = dl.Polygon(positions=[[57, 10], [57, 11], [56, 11], [57, 10]])
# Some tile urls.
keys = ["watercolor", "toner", "terrain"]
url_template = "http://{{s}}.tile.stamen.com/{}/{{z}}/{{x}}/{{y}}.png"
attribution = 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, ' \
              '<a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data ' \
              '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'

app.layout = html.Div(dl.Map([
   
    dl.LayersControl(
        [dl.BaseLayer(dl.TileLayer(url=url_template.format(key), attribution=attribution),
                      name=key, checked=key == "toner") for key in keys] +
        [dl.Overlay(dl.LayerGroup(markers), name="markers", checked=True),
         dl.Overlay(dl.LayerGroup(polygon), name="polygon", checked=True)],
         id="map"
    ), dl.LayerGroup(id="layer"),
], zoom=7, center=(56, 10)), style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})


@app.callback(Output("layer", "children"), [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip("({:.3f}, {:.3f})".format(*click_lat_lng)))]

if __name__ == '__main__':
    app.run_server(mode="inline")